In [ ]:

import utils.common
import evaluation
import importlib
import numpy as np
import time
import models.MatrixFactorization
import models.ConstraintAutoRec
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.MatrixFactorization)
importlib.reload(models.ConstraintAutoRec)



In [126]:
model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=10, novelty_weight=0.1, diversity_weight=0.1, latent_dim=64)
model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


Epoch 1/10
19/19 [==============================] - 8s 404ms/step - loss: 0.2570 - accuracy: 0.0146
Epoch 2/10
19/19 [==============================] - 0s 14ms/step - loss: 0.1808 - accuracy: 0.0477
Epoch 3/10
19/19 [==============================] - 0s 16ms/step - loss: 0.1677 - accuracy: 0.0536
Epoch 4/10
19/19 [==============================] - 0s 15ms/step - loss: 0.1537 - accuracy: 0.0415
Epoch 5/10
19/19 [==============================] - 0s 15ms/step - loss: 0.1487 - accuracy: 0.0230
Epoch 6/10
19/19 [==============================] - 0s 15ms/step - loss: 0.1383 - accuracy: 0.0230
Epoch 7/10
19/19 [==============================] - 0s 13ms/step - loss: 0.1344 - accuracy: 0.0328
Epoch 8/10
19/19 [==============================] - 0s 14ms/step - loss: 0.1245 - accuracy: 0.0242
Epoch 9/10
19/19 [==============================] - 0s 14ms/step - loss: 0.1250 - accuracy: 0.0197
Epoch 10/10
19/19 [==============================] - 0s 16ms/step - loss: 0.1103 - accuracy: 0.0296


In [ ]:
diversity_model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=30, novelty_weight=0.25, diversity_weight=2.0)
diversity_model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


In [ ]:
def create_model(novelty_weight):
    m = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=15, novelty_weight=novelty_weight, diversity_weight=0, batch_size=128)
    m.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])
    return m

nov_models2 = [create_model(x) for x in [0.1, 0.5, 1.0, 2.0]]

In [ ]:
import matplotlib.pyplot as plt
models

In [ ]:
#ev = evaluation.Evaluation(utils.common.movie_lens)
#big_eval = ev.evaluate_models(models)
big_eval.head()

In [ ]:
big_eval

In [ ]:
plt.bar(eval_data.index, eval_data['epc@10'])

In [ ]:
plt.plot(eval_data['novelty_weight'], eval_data['epc@10'], 'ro', ls='--')
plt.plot(eval_data['novelty_weight'], eval_data['accuracy'], 'bo', ls='--')
plt.show()

In [ ]:
def plot_bar(data, axis, x_param):
    bottom = data[axis].min() 
    top = data[axis].max()
    margin = (top - bottom) * 0.05
    fig, ax = plt.subplots()
    ax.bar(data.index, data[axis] )
    ax.set_xticks(data.index)
    ax.set_xticklabels(data[x_param])
    ax.set_ylabel(axis)
    ax.set_xlabel(x_param)
    plt.show()

In [ ]:
plot_bar(eval_data, 'epc@10', 'novelty_weight')

In [ ]:
plot_bar(eval_data, 'accuracy', 'novelty_weight')

In [129]:

import utils.common
import evaluation
import importlib
import numpy as np
import time
import models.MatrixFactorization
import models.BPR
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.MatrixFactorization)
importlib.reload(models.BPR)



<module 'models.BPR' from '/mnt/e/owncloud/FH/MasterThesis/Code/models/BPR.py'>

In [ ]:
# mf = models.MatrixFactorization.MatrixFactorization(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=10, latent_dim = 64)
# mf.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])

bpr = models.BPR.BPR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=5, latent_dim = 32, batch_size=128)
bpr.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])

In [125]:
ev = evaluation.Evaluation(utils.common.movie_lens)
ev.evaluate(bpr)

Evaluating BPR |============================================================>| 100.0% 


{'accuracy': 0.728796169630643,
 'precision': 0.8695271453590193,
 'recall': 0.8004836759371221,
 'map@1': 0.19327731092436976,
 'map@5': 0.08470121381886088,
 'map@10': 0.06565318984736752,
 'diversity@5': 0.30614628595624477,
 'diversity@10': 0.31488377299134696,
 'epc@5': 0.7644980470235259,
 'epc@10': 0.7767215907725664,
 'epd@5': 0.33222268291415785,
 'name': 'BPR',
 'latent_dim': 32,
 'epochs': 10,
 'batch_size': 128}

In [127]:
result = ev.evaluate_models([model, bpr])

Evaluating ConstraintAutoRec |============================================================>| 100.0% 
Evaluating BPR |============================================================>| 100.0% 


In [ ]:
ev.evaluate_models(nov_models2)

In [128]:
result

,accuracy,accuracy_weight,batch_size,dimensions,diversity@10,diversity@5,diversity_weight,epc@10,epc@5,epd@5,...,latent_dim,latent_dims,map@1,map@10,map@5,name,novelty_weight,optimizer,precision,recall
0,0.672709,1.0,32,10379.0,0.234590,0.225265,0.1,0.735680,0.716488,0.316391,...,NaN,64.0,0.134454,0.045434,0.056975,ConstraintAutoRec,0.1,adam,0.894410,0.696493
1,0.728796,NaN,128,NaN,0.314884,0.306146,NaN,0.776722,0.764498,0.332223,...,32.0,NaN,0.193277,0.065653,0.084701,BPR,NaN,NaN,0.869527,0.800484
